In [1]:
import shutil
from pathlib import Path
import numpy as np 
import pandas as pd
import imageio.v3 as iio
import matplotlib.pyplot as plt

import zipfile
from tqdm import tqdm
#from tqdm.notebook import tqdm

from numpy.lib.stride_tricks import sliding_window_view
from mpl_toolkits.axes_grid1 import ImageGrid

from datasets.urban_footprint_extraction import (
    transformation_strategy_cityosm,
    transformation_strategy_potsdam,
    transformation_strategy_vaihingen
)

In [2]:
CITIES_LATLONG = {
        "austin": (30.265842917057622, -97.74755465239008),
        "chicago": (41.874177664952654, -87.63999821844027),
        "vaihingen": (48.73014367567042, 9.104438440261527),
        "tokyo": (35.864256223966, 139.7227349836842),
        "kitsap": (47.69255762120212, -122.67612807263788),
        "tyrol-w": (47.26856603333054, 11.405558814052391),
        "vienna": (48.21066303665396, 16.378259010071375),
        "potsdam": (52.38999159662546, 13.062138562210476),
        "berlin": (52.52005606083578, 13.413606124150403),
        "zurich": (47.37592867075607, 8.544278306670433),
        "paris": (48.85512324534345, 2.3483982735980855),
    }

In [3]:
DATA = Path.home() / "datasets" / "urban-footprint"
DATASET_ZIPFILES = ("berlin", "paris", "zurich", "tokyo", "chicago", "inria", "vaihingen", "potsdam")

In [5]:
def get_tiled_view(image: np.ndarray, kernel: tuple[int, int, int]):
    """
    Return a non-overlapping sliding window view of image 

    image.shape: (height, width, num_channels)
    kernel.shape: (kernel_height, kernel_width, num_kernel_channels) 

    return.shape (#Tiles, K_H, K_W, K_C)
    """
    return (
        sliding_window_view(image, kernel[:2], (0,1)) #type: ignore
        [::kernel[0], ::kernel[1]]
        .reshape(-1, kernel[2], kernel[0], kernel[1])
        .transpose(0, 2, 3, 1)
        .squeeze()
    )

In [6]:
def tile_dataset(tile: tuple[int,int]):
    filenames = sorted([f.name for f in IMAGE_DIR.iterdir()])
    for filename in tqdm(filenames):
        images = get_tiled_view(
            image = iio.imread(IMAGE_DIR/filename).squeeze(),
            kernel = (tile[0], tile[1], 3)
        )
        masks = get_tiled_view(
            image = iio.imread(MASK_DIR/filename).squeeze(),
            kernel = (tile[0], tile[1], 1)
        )
        name = filename.split(".")[0]
        extn = filename.split(".")[-1]
        try:
            for idx, (image, mask) in enumerate(zip(images, masks)):
                iio.imwrite(TILED_IMAGE_DIR/f"{name}_{idx}.{extn}", image) #type: ignore
                iio.imwrite(TILED_MASK_DIR/f"{name}_{idx}.{extn}", mask) #type: ignore
        except:
            print(filename)
            continue

Train Val Test Splits
1. Random Split: Each location will be split based on the test_split and val_split parameters
2. Continental Split: Train on Europe, Test on NA or Vice Versa 
3. Cultural Split: Train on Developed Locations Like Paris, Chicago and Zurich and Test on Rawanda, Kenya and Rio  
4. Unsupervised Split: Unsupervised Training on Inria-Test and Finetune on Inria-Train (with varying fractions of training data)

Convert individual image datasets into .h5 or .npy files, dynamically load cropped images at runtime using the right dataset class. This will avoid saving multiple copies of the dataset on disk for different tile sizes. On the downside, will have to define a function to save projection information into the .h5 dataset, also will make visualization in GIS much harder.

In [ ]:
import zipfile
import h5py

DATASET = DATA / "NEW2-AerialImageDataset.zip"
UNSUPERVISED_LOCATIONS = ("bellingham", "bloomington", "innsbruck", "sfo", "tyrol-e")
SUPERVISED_LOCATIONS = ("austin", "chicago", "kitsap", "vienna", "tyrol-w")

sup_filenames = tuple((f"{x}{num}.tif" for x in SUPERVISED_LOCATIONS for num in range(1, 37)))
unsup_filenames = tuple((f"{x}{num}.tif" for x in UNSUPERVISED_LOCATIONS for num in range(1, 37)))
with zipfile.ZipFile(DATASET) as zf:
    for filename in tqdm(sup_filenames):
        image = iio.imread(DATASET/"AerialImageDataset"/"train"/"images"/filename)
        mask = iio.imread(DATASET/"AerialImageDataset"/"train"/"gt"/filename)
        break